# Math 4383 Project
### Zachary Koenig, Colin Crippen

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
# --- Load the 8x8 digits dataset ---
digits = load_digits()

# The dataset contains 8x8 images and their corresponding digit labels.
# X contains image data and y contains the target labels.
X = digits.images      # Shape: (n_samples, 8, 8)
y = digits.target      # Shape: (n_samples,)

# --- Data Preprocessing ---

# 1. Flatten the images:
#    Convert each 8x8 image into a 1D array of 64 pixels.
n_samples = X.shape[0]
X_flat = X.reshape(n_samples, -1)   # Now shape is (n_samples, 64)

# 2. Normalize the inputs:
#    The pixel values in the digits dataset are in the range 0-16.
#    Dividing by 16 scales them to the range [0, 1].
X_normalized = X_flat / 16.0

# 3. One-Hot Encode the Labels:
#    There are 10 classes (digits 0-9), so create one-hot encoded vectors.
num_classes = 10
y_onehot = np.eye(num_classes)[y]

# --- Split into Training and Verification (Validation) Sets ---
# For example, reserve 20% of the data for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_normalized, y_onehot, test_size=0.2, random_state=42
)

# Verify the shapes
print("Training set shape:", X_train.shape, y_train.shape)  # Should show ~80% of the samples
print("Validation set shape:", X_val.shape, y_val.shape)      # Should show ~20% of the samples
print("One-hot encoded y shape:", y_onehot.shape)    # Expected: (n_samples, 10)

Training set shape: (1437, 64) (1437, 10)
Validation set shape: (360, 64) (360, 10)
One-hot encoded y shape: (1797, 10)


Sigmoid function to enable nonlinear function representation

In [3]:
# --- Activation functions and their derivatives ---

def sigmoid(x):
    """Compute the sigmoid activation."""
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    """Compute the derivative of the sigmoid function."""
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    """Compute the softmax activation for each row of x."""
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))  # for numerical stability
    return exps / np.sum(exps, axis=1, keepdims=True)

In [40]:
# --- Hyperparameters ---
input_size = 8 * 8   # 64 inputs, one per pixel
hidden_size = 64       # You can adjust this
output_size = 10       # 10 classes for digits 0-9

learning_rate = 0.1
epochs = 1_000

# --- Initialize Weights and Biases ---
np.random.seed(42)  # For reproducibility
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

In [41]:
# --- Training Loop ---
num_train_samples = X_train.shape[0]  # Use real number of training samples
for epoch in range(epochs):
    # Forward pass:
    Z1 = np.dot(X_train, W1) + b1       # Input to hidden layer
    A1 = sigmoid(Z1)                   # Hidden layer activation
    Z2 = np.dot(A1, W2) + b2            # Hidden to output layer
    A2 = sigmoid(Z2)                   # Output activation

    # Compute Cross-Entropy Loss (adding a small constant for numerical stability)
    loss = -np.mean(np.sum(y_train * np.log(A2 + 1e-8), axis=1))
    
       # Backward pass:
    # For softmax and cross-entropy, the gradient simplifies:
    dZ2 = A2 - y_train
    dW2 = np.dot(A1.T, dZ2) / X_train.shape[0]
    db2 = np.sum(dZ2, axis=0, keepdims=True) / X_train.shape[0]
    
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_deriv(Z1)
    dW1 = np.dot(X_train.T, dZ1) / X_train.shape[0]
    db1 = np.sum(dZ1, axis=0, keepdims=True) / X_train.shape[0]
    
    # Update the weights and biases using gradient descent:
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: loss = {loss:.4f}")


# Save the trained weights and biases to disk
np.save('W1.npy', W1)
np.save('b1.npy', b1)
np.save('W2.npy', W2)
np.save('b2.npy', b2)

Epoch 0: loss = 0.7012
Epoch 100: loss = 2.3000
Epoch 200: loss = 2.2964
Epoch 300: loss = 2.2868
Epoch 400: loss = 2.2602
Epoch 500: loss = 2.1891
Epoch 600: loss = 2.0295
Epoch 700: loss = 1.7801
Epoch 800: loss = 1.5283
Epoch 900: loss = 1.3305


Test a single input:

# Load the saved weights and biases
```python
W1 = np.load('W1.npy')
b1 = np.load('b1.npy')
W2 = np.load('W2.npy')
b2 = np.load('b2.npy')

x_new = np.load('test')
A1_new = sigmoid(np.dot(x_new, W1) + b1)
A2_new = sigmoid(np.dot(A1_new, W2) + b2)
prediction = np.argmax(A2_new)
print("Predicted digit:", prediction)
```

In [42]:
# --- Perform Predictions on the Test Set Using the Loaded Weights ---
# Forward pass:
# 1. Compute activations for the hidden layer.
Z1 = np.dot(X_val, W1) + b1   # Linear combination for the hidden layer.
A1 = sigmoid(Z1)              # Apply sigmoid activation.

# 2. Compute activations for the output layer.
Z2 = np.dot(A1, W2) + b2       # Linear combination for the output layer.
A2 = sigmoid(Z2)              # Output activations.

# The predicted class for each sample is the one with the highest activation.
predictions = np.argmax(A2, axis=1)
true_labels = np.argmax(y_val, axis=1)

# Calculate the accuracy of the model on the test set.
accuracy = np.mean(predictions == true_labels)
print("Test set accuracy: {:.2f}%".format(accuracy * 100))

Test set accuracy: 78.06%
